# Análisis de Patrones de Correos
Este notebook descarga correos históricos para identificar casos borde en el parser.

In [ ]:
import pandas as pd
from src.ingestion import GmailClient
from src.parser import TransactionParser
import logging

# Configurar pandas para ver todo el texto
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# 1. Inicializar Cliente
client = GmailClient()
parser = TransactionParser()

print("Autenticación exitosa.")

In [ ]:
# 2. Descargar Historial (últimos 50 correos de Bancolombia)
# Ajusta el 'sender' o el 'custom_query' según necesites
# Ejemplo: "from:alertasynotificaciones@bancolombia.com.co" o "subject:Transferencia"

SEARCH_QUERY = "Bancolombia"  # Busca "Bancolombia" en general
MAX_EMAILS = 50

print(f"Buscando últimos {MAX_EMAILS} correos con query: '{SEARCH_QUERY}'...")

emails = client.fetch_unread_emails(
    max_results=MAX_EMAILS, 
    custom_query=SEARCH_QUERY
)

print(f"Descargados: {len(emails)}")

In [ ]:
# 3. Probar el Parser
results = []

for email in emails:
    # Preferimos el cuerpo, fallback al snippet
    text = email.get('body') or email.get('snippet', '')
    
    parsed = parser.parse(text)
    
    results.append({
        "id": email['id'],
        "date_extracted": parsed['date'],
        "amount": parsed['amount'],
        "merchant": parsed['merchant'],
        "original_snippet": email.get('snippet', '')[:100] + "...",
        "full_text": text[:200]  # Guardamos primeros 200 chars para análisis
    })

df = pd.DataFrame(results)

In [ ]:
# 4. Análisis de Casos Fallidos (Monto 0 o Merchant UNKNOWN)
failed_cases = df[ (df['amount'] == 0) | (df['merchant'] == 'UNKNOWN') ]

print(f"Total procesados: {len(df)}")
print(f"Casos exitosos: {len(df) - len(failed_cases)}")
print(f"Casos fallidos/dudosos: {len(failed_cases)}")

# Mostrar los fallidos para inspeccionar
print("\n--- CASOS QUE REQUIEREN ATENCIÓN ---")
display(failed_cases[['amount', 'merchant', 'full_text']])

In [ ]:
# 5. Exportar para revisión manual si deseas
# df.to_csv("analisis_correos.csv", index=False)